In [14]:
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as nafc

from nlpaug.util import Action

In [21]:
from tqdm.notebook import tqdm

In [2]:
text = 'The quick brown fox jumps over the lazy dog .'
print(text)

The quick brown fox jumps over the lazy dog .


In [1]:
import torch
import torch.nn.functional as F
from torchtext import data
from torchtext import datasets
import time
import random

In [22]:
RANDOM_SEED = 123
torch.manual_seed(RANDOM_SEED)

VOCABULARY_SIZE = 20000
LEARNING_RATE = 1e-4
BATCH_SIZE = 128
NUM_EPOCHS = 15
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

EMBEDDING_DIM = 128
HIDDEN_DIM = 256
OUTPUT_DIM = 1

In [3]:
import numpy as np

In [4]:
def random_deletion(tokens, p=0.5):
    if len(tokens) == 0:
        return tokens

    mask = np.random.rand(len(tokens)) > p
    remaining_tokens = list(np.array(tokens)[mask])
    # remaining_tokens = [token for token in tokens if random.uniform(0, 1) > p]
    if len(remaining_tokens) == 0:
        return [random.choice(tokens)]  # 如果全部删除，则随机保留一个
    return remaining_tokens

# aug = nas.AbstSummAug(model_path='t5-base', device='cuda')
    
def t5_sum(tokens, aug, n=1):
    text = ' '.join(tokens)
    augmented_text = aug.augment(text, n)
    
    return augmented_text

In [5]:
TEXT = data.Field(tokenize='spacy',
                  include_lengths=True) # necessary for packed_padded_sequence
LABEL = data.LabelField(dtype=torch.float)
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)
train_data, valid_data = train_data.split(random_state=random.seed(RANDOM_SEED),
                                          split_ratio=0.8)

print(f'Num Train: {len(train_data)}')
print(f'Num Valid: {len(valid_data)}')
print(f'Num Test: {len(test_data)}')

Num Train: 20000
Num Valid: 5000
Num Test: 25000


In [33]:
text = ' '.join(train_data.examples[0].text)

In [35]:
aug = naw.SynonymAug(aug_src='ppdb', model_path='./nlpaug_model/ppdb-2.0-tldr')

In [37]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/xiyang/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [22]:
new_examples = []
# aug = nas.AbstSummAug(model_path='t5-base', device='cuda')
# aug = naw.SynonymAug(aug_src='ppdb', model_path='./nlpaug_model/ppdb-2.0-tldr')
aug = naw.ContextualWordEmbsAug(model_path='bert-base-uncased', action="substitute")
for example in tqdm(train_data.examples):
    # 对原始文本进行随机删除数据增强
    # example_augmented_text = random_deletion(example.text)
    example_augmented_text = t5_sum(example.text, aug, 1)
    
    # 创建一个新的例子并将其添加到训练数据中
    new_example = data.Example.fromlist([example_augmented_text, example.label], [('text', TEXT), ('label', LABEL)])
    new_examples.append(new_example)
train_data.examples.extend(new_examples)

  0%|          | 0/20000 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [14]:
len(train_data.examples)

40000

In [13]:
import torchtext.vocab as vocab

TEXT.build_vocab(train_data,
                 max_size=VOCABULARY_SIZE,
                 # vectors='glove.6B.100d',
                 vectors=vocab.GloVe(name='6B', dim=100),
                 unk_init=torch.Tensor.normal_)
LABEL.build_vocab(train_data)

print(f'Vocabulary size: {len(TEXT.vocab)}')
print(f'Number of classes: {len(LABEL.vocab)}')

Vocabulary size: 20002
Number of classes: 2


In [24]:
def custom_collate(batch):
    # 将文本数据转换为对应的词嵌入
    text, labels = zip(*batch)
    text = [TEXT.vocab.vectors[TEXT.vocab.stoi[word]] for sentence in text for word in sentence]
    text = torch.stack(text)
    
    labels = torch.tensor(labels)
    
    return text, labels

In [25]:
train_loader, valid_loader, test_loader = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size=BATCH_SIZE,
    sort_within_batch=True, # necessary for packed_padded_sequence
    collate_fn=custom_collate,
    device=DEVICE)

TypeError: Iterator.__init__() got an unexpected keyword argument 'collate_fn'

In [15]:
print('Train')
for batch in train_loader:
    print(f'Text matrix size: {batch.text[0].size()}')
    print(f'Target vector size: {batch.label.size()}')
    print(batch.text[0])
    break
    
print('\nValid:')
for batch in valid_loader:
    print(f'Text matrix size: {batch.text[0].size()}')
    print(f'Target vector size: {batch.label.size()}')
    break
    
print('\nTest:')
for batch in test_loader:
    print(f'Text matrix size: {batch.text[0].size()}')
    print(f'Target vector size: {batch.label.size()}')
    break

Train
Text matrix size: torch.Size([133, 128])
Target vector size: torch.Size([128])
tensor([[18453,    11,   368,  ...,   170,    14,   359],
        [  911,   233,     2,  ...,    31,    55,     7],
        [    7,    36,   144,  ...,   203,    19,     2],
        ...,
        [  224,     5,  2343,  ...,    39,     4,     4],
        [   42,  5780,   663,  ...,     1,     1,     1],
        [    4,     4,     4,  ...,     1,     1,     1]], device='cuda:0')

Valid:
Text matrix size: torch.Size([61, 128])
Target vector size: torch.Size([128])

Test:
Text matrix size: torch.Size([42, 128])
Target vector size: torch.Size([128])


In [18]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.embedding.weight.data.copy_(TEXT.vocab.vectors)

        self.rnn = nn.LSTM(embedding_dim, hidden_dim)

        if output_dim is not None:
            self.fc = nn.Linear(hidden_dim, output_dim)
        else:
            self.fc = None
        
    def forward(self, text, text_length):

        #[sentence len, batch size] => [sentence len, batch size, embedding size]
        embedded = self.embedding(text)
        
        packed = torch.nn.utils.rnn.pack_padded_sequence(embedded, text_length.cpu()).cuda()
        
        #[sentence len, batch size, embedding size] => 
        #  output: [sentence len, batch size, hidden size]
        #  hidden: [1, batch size, hidden size]
        packed_output, (hidden, cell) = self.rnn(packed)

        if self.fc is not None:
            return self.fc(hidden.squeeze(0)).view(-1)
        else:
            return hidden.squeeze(0)

In [23]:
INPUT_DIM = len(TEXT.vocab)

torch.manual_seed(RANDOM_SEED)
model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)
model = model.to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

RuntimeError: The size of tensor a (128) must match the size of tensor b (100) at non-singleton dimension 1

In [20]:
def compute_binary_accuracy(model, data_loader, device):
    model.eval()
    correct_pred, num_examples = 0, 0
    with torch.no_grad():
        for batch_idx, batch_data in enumerate(data_loader):
            text, text_lengths = batch_data.text
            logits = model(text, text_lengths)
            predicted_labels = (torch.sigmoid(logits) > 0.5).long()
            num_examples += batch_data.label.size(0)
            correct_pred += (predicted_labels == batch_data.label.long()).sum()
        return correct_pred.float()/num_examples * 100


In [21]:
start_time = time.time()

for epoch in range(NUM_EPOCHS):
    model.train()
    for batch_idx, batch_data in enumerate(train_loader):
        
        text, text_lengths = batch_data.text
        
        ### FORWARD AND BACK PROP
        logits = model(text, text_lengths)
        cost = F.binary_cross_entropy_with_logits(logits, batch_data.label)
        optimizer.zero_grad()
        
        cost.backward()
        
        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        
        ### LOGGING
        if not batch_idx % 50:
            print (f'Epoch: {epoch+1:03d}/{NUM_EPOCHS:03d} | '
                   f'Batch {batch_idx:03d}/{len(train_loader):03d} | '
                   f'Cost: {cost:.4f}')

    with torch.set_grad_enabled(False):
        print(f'training accuracy: '
              f'{compute_binary_accuracy(model, train_loader, DEVICE):.2f}%'
              f'\nvalid accuracy: '
              f'{compute_binary_accuracy(model, valid_loader, DEVICE):.2f}%')
        
    print(f'Time elapsed: {(time.time() - start_time)/60:.2f} min')
    
print(f'Total Training Time: {(time.time() - start_time)/60:.2f} min')
print(f'Test accuracy: {compute_binary_accuracy(model, test_loader, DEVICE):.2f}%')

Epoch: 001/015 | Batch 000/157 | Cost: 0.6946
Epoch: 001/015 | Batch 050/157 | Cost: 0.6873
Epoch: 001/015 | Batch 100/157 | Cost: 0.6821
Epoch: 001/015 | Batch 150/157 | Cost: 0.5984
training accuracy: 67.05%
valid accuracy: 68.88%
Time elapsed: 0.09 min
Epoch: 002/015 | Batch 000/157 | Cost: 0.5622
Epoch: 002/015 | Batch 050/157 | Cost: 0.6012
Epoch: 002/015 | Batch 100/157 | Cost: 0.5809
Epoch: 002/015 | Batch 150/157 | Cost: 0.7197
training accuracy: 52.97%
valid accuracy: 52.88%
Time elapsed: 0.18 min
Epoch: 003/015 | Batch 000/157 | Cost: 0.6832
Epoch: 003/015 | Batch 050/157 | Cost: 0.6491
Epoch: 003/015 | Batch 100/157 | Cost: 0.6330
Epoch: 003/015 | Batch 150/157 | Cost: 0.6765
training accuracy: 65.39%
valid accuracy: 66.08%
Time elapsed: 0.26 min
Epoch: 004/015 | Batch 000/157 | Cost: 0.6503
Epoch: 004/015 | Batch 050/157 | Cost: 0.5238
Epoch: 004/015 | Batch 100/157 | Cost: 0.5256
Epoch: 004/015 | Batch 150/157 | Cost: 0.5892
training accuracy: 76.24%
valid accuracy: 76.26%

## No Aug

training accuracy: 90.28%
valid accuracy: 84.80%
Time elapsed: 1.28 min
Total Training Time: 1.28 min
Test accuracy: 84.42%

## Rand Del
training accuracy: 90.51%
valid accuracy: 85.56%
Time elapsed: 1.99 min
Total Training Time: 1.99 min
Test accuracy: 85.02%

## Synonym Aug
training accuracy: 66.77%
valid accuracy: 80.44%
Time elapsed: 1.50 min
Total Training Time: 1.50 min
Test accuracy: 79.55%

In [11]:
import spacy
nlp = spacy.load('en')

def predict_sentiment(model, sentence):
    # based on:
    # https://github.com/bentrevett/pytorch-sentiment-analysis/blob/
    # master/2%20-%20Upgraded%20Sentiment%20Analysis.ipynb
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(DEVICE)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.sigmoid(model(tensor, length_tensor))
    return prediction.item()

In [16]:


print('Probability positive:')
predict_sentiment(model, "Do i really love this movie? Yes I do")



Probability positive:


0.5735723376274109